# Analyse and run the model

In this notebook we are going to build, test, analyze and compare the model with the previous versions. This is followed by improvements to the model and the data. We run this cycle a few times until we achieve realistic and nice results.

But first, let's start by importing the necessary libraries and files, as follows:

In [ ]:
# To read the data 
import pickle
import os
import _pickle as cPickle
import bz2

# Import the file with the model functions
import sys
sys.path.append('/Users/marya/PycharmProjects/EmpathicRobot')
from conv_model import *
from models.functions import *

The structure of the models can still be found [here](https://github.com/BB8-2020/EmpathicRobot/blob/classification-model/models/classification-model/conv/conv_model.py).

## Read data

__FerPlus__

As we have done before, our data is ready to use. In this section we will first use **ferPlus** to train the model. After that we would us the **AffectNet** to train the model. This data has already been read, prepared and stored in **hier linkje zetten** this file. 
For now, our data is in a pickel file that we will read as follows:

For simplicity, we set up the path to the data as follows, you can also set it to your own path.

In [ ]:
os.chdir(os.getcwd() + '/data/')

We immediately split the data into train, test and validation set.

In [ ]:
x_trainf, y_trainf, x_valf, y_valf, x_testf, y_testf = cPickle.load(bz2.BZ2File('ferPlus_processed', 'rb'))

As we see, the data consists of train set that contains 80% of the data. The validation and the test set are equal in size 20% and are used to subsequently test the model.

This data has already been cleaned and normalized so we don't have to do anything with the data anymore.



In [ ]:
print(f"Train set:\nX_train shape:{x_trainf.shape}\nY_train shape:{y_trainf.shape}\n")

print(f"Test set:\nX_test shape:{x_testf.shape}\nY_test shape:{y_testf.shape}\n")

print(f"Validation set:\nX_val shape:{x_valf.shape}\nY_val shape:{y_valf.shape}")

__AffectNet__

We do the same as the FerPlus. So we split the data first and print the shape out. 

In [ ]:
x_traina, y_traina, x_vala, y_vala, x_testa, y_testa = cPickle.load(bz2.BZ2File('affectNet_processed', 'rb'))

In [ ]:
print(f"Train set:\nX_train shape:{x_trainf.shape}\nY_train shape:{y_trainf.shape}\n")

print(f"Test set:\nX_test shape:{x_testf.shape}\nY_test shape:{y_testf.shape}\n")

print(f"Validation set:\nX_val shape:{x_valf.shape}\nY_val shape:{y_valf.shape}")

## Model version 1

### Build model

Now it is finally time to start working on the model.
We are going to start with the following model:

### Creat model

__1. Conv Layer__

The first layer consists of 64 3x3 filters with ReLU. We set the input of this layer equal to the shape of the train data, which is (48, 48, 1).
We leave the stride and padding at the default value. We do add a Batch normalization. The output of this layer (the activation 
map) is (46, 46, 64).
As laste we add a dropout of 0.5. That brings us to a result of (46, 46, 64)

__2. Conv Layer__


The second layer consists of 64 filters of 3x3 and here we apply relu as well. The output of this layer woudld be (44, 44, 64). we also apply a max pooling of (2,2) and strides od (2,2) that produces an output shape of (44, 44, 64).

__3. Conv Layer__


This layer consists of 128 filter of 3x3 we also apply relu here. The output of this layer would be (20, 20, 128). We do add  Batch normalization. The output of this layer is (20, 20, 128).

__4. Conv Layer__


This layer consists of 128 filter of 3x3 we also apply relu here. The output of this layer would be (18, 18, 128). We also apply a max pooling of (2,2) and strides od (2,2) that produces an output shape of (9, 9, 128).

__5. Conv Layer__ 


The last layer consists of 256 fliter of 3x3 and we apply reule. The output would be (7, 7, 256). We apply here also a max pooling of (2,2) and strides od (2,2) that produces an output shape of (7, 7, 256).
To this layer we add a flatten option, that means the output shape of this layer would be (12544)

__3. Fully connected layer__


This layer takes (12544) as input. We apply a droupout of 0.2. The next dense layer teaks (1024) as input. After applying the drouput for the last time the output shape of this layer is the probability of 7 classes.

In [ ]:
# We create all the models that we got 
models = build_models(input_shape=(48, 48, 1), num_classes=7)

In [ ]:
# Build the sequential model version 1
model1 = Sequential(models[0]['layers'], name = models[0]['name'])

Let's check the summary out:

In [ ]:
model1.summary()

Looks good, time to compile!


### Model compile and train


To compile the model we use Adam optimaizer and binary crossentropy as los function. Let us now train the model.

In [ ]:
# compile the model
model1.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

We set the epochs to 100 and the batch size tot 64.

In [ ]:
epochs = 100
batch_size = 64

Let's train the model! 

_Train with ferPlus_

In [ ]:
history_fer = model1.fit(x_trainf, y_trainf, batch_size=batch_size, epochs=epochs,
                        steps_per_epoch=len(x_trainf) // batch_size,
                        validation_data=(x_valf, y_valf), verbose=2)

Now we're going to test our model using the test set for the model.



In [ ]:
test_loss_fer, test_acc_fer = model1.evaluate(x_testf, y_testf, batch_size=batch_size)
print(f"Test loss: {test_loss_fer:.4f}")
print(f"Test accuracy: {test_loss_fer:.4f}")

In [ ]:
## bespreek de resultaten van deze train

_Train with AffectNet_

As we saw above, the results are not too great. Therefore we will now try to adjust the settings of the model .

In [ ]:
history_affect = model1.fit(x_traina, y_traina, batch_size=batch_size, epochs=epochs,
                        steps_per_epoch=len(x_traina) // batch_size,
                        validation_data=(x_vala, y_vala), verbose=2)

In [ ]:
test_loss_affect, test_acc_affect = model1.evaluate(x_testa, y_testa, batch_size=batch_size)
print(f"Test loss: {test_loss_affect:.4f}")
print(f"Test accuracy: {test_acc_affect:.4f}")

In [ ]:
## bespreek de resultaten van deze train

## Results 

Let's discuss the results of the model.

In [ ]:
plot_acc_loss(history_fer)

In [ ]:
plot_acc_loss(history_affect)

In [ ]:
# TODO: BESPREKK DE RESULTATEN!!!! MAAR EERST MOET HET MODEL GETRAIEND WORDEN

## Model version 2

In [ ]:
# TODO, fix wat hieronder staat!!!

### Build model

#### Create model

__1. Conv Layer__

The first layer consists of 64 3x3 filters with ReLU. We set the input of this layer equal to the shape of the train data, which is (48, 48, 1).
We leave the stride and padding at the default value. We do add a Batch normalization. The output of this layer (the activation 
map) is (46, 46, 64).
As laste we add a dropout of 0.5. That brings us to a result of (46, 46, 64)

__2. Conv Layer__


The second layer consists of 64 filters of 3x3 and here we apply relu as well. The output of this layer woudld be (44, 44, 64). we also apply a max pooling of (2,2) and strides od (2,2) that produces an output shape of (44, 44, 64).

__3. Conv Layer__


This layer consists of 128 filter of 3x3 we also apply relu here. The output of this layer would be (20, 20, 128). We do add  Batch normalization. The output of this layer is (20, 20, 128).

__4. Conv Layer__


This layer consists of 128 filter of 3x3 we also apply relu here. The output of this layer would be (18, 18, 128). We also apply a max pooling of (2,2) and strides od (2,2) that produces an output shape of (9, 9, 128).

__5. Conv Layer__ 


The last layer consists of 256 fliter of 3x3 and we apply reule. The output would be (7, 7, 256). We apply here also a max pooling of (2,2) and strides od (2,2) that produces an output shape of (7, 7, 256).
To this layer we add a flatten option, that means the output shape of this layer would be (12544)

__3. Fully connected layer__


This layer takes (12544) as input. We apply a droupout of 0.2. The next dense layer teaks (1024) as input. After applying the drouput for the last time the output shape of this layer is the probability of 7 classes.

In [ ]:
model2 = Sequential(models[1]['layers'], name = models[1]['name'])

In [ ]:
model2.summary()

### Model compile and train


In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

_Train with ferPlus_

In [ ]:
history_fer = model2.fit(x_trainf, y_trainf, batch_size=batch_size, epochs=epochs,
                        steps_per_epoch=len(x_trainf) // batch_size,
                        validation_data=(x_valf, y_valf), verbose=2)

In [ ]:
test_loss_fer, test_acc_fer = model1.evaluate(x_testf, y_testf, batch_size=batch_size)
print(f"Test loss: {test_loss_fer:.4f}")
print(f"Test accuracy: {test_acc_fer:.4f}")

_Train with AffectNet_

In [ ]:
history_affect = model2.fit(x_traina, y_traina, batch_size=batch_size, epochs=epochs,
                        steps_per_epoch=len(x_traina) // batch_size,
                        validation_data=(x_vala, y_vala), verbose=2)

In [ ]:
test_loss_affect, test_acc_affect = model1.evaluate(x_testa, y_testa, batch_size=batch_size)
print(f"Test loss: {test_loss_affect:.4f}")
print(f"Test accuracy: {test_acc_affect:.4f}")

### Results 

In [ ]:
plot_acc_loss(history_fer)

In [ ]:
plot_acc_loss(history_affect)

In [ ]:
# TODO: BESPREKK DE RESULTATEN!!!! MAAR EERST MOET HET MODEL GETRAIEND WORDEN

The results are going beter, next we are going to try to add some argumentation to the data. That could help our model to leren more. You can find the file where the data has been argumendated right [hier](linkjeee) __fix it!__

_FerPlus_

We split the data again

In [ ]:
datagen,x_train_arg, y_train_arg, x_val_arg, y_val_arg, x_test_arg, y_test_arg =
                                                                            cPickle.load(bz2.BZ2File('ferPlus_augment', 'rb'))

Now we are going to just fit the model using this data.

In [ ]:
history_fer = model2.fit(datagen.flow(x_train_arg, y_train_arg, batch_size=batch_size), epochs=epochs,
                            steps_per_epoch=len(x_train_arg) // batch_size,
                            validation_data=(x_val_arg, y_val_arg), verbose=2)

In [ ]:
test_loss_fer, test_acc_fer = model2.evaluate(x_test_arg, y_test_arg, batch_size=batch_size)
print(f"Test loss: {test_loss_fer:.4f}")
print(f"Test accuracy: {test_acc_fer:.4f}")

_AffectNet_

In [ ]:
datagen,x_train_arg, y_train_arg, x_val_arg, y_val_arg, x_test_arg, y_test_arg =
                                                                            cPickle.load(bz2.BZ2File('affectNet_augment', 'rb'))

In [ ]:
history_affect = model2.fit(datagen.flow(x_train_arg, y_train_arg, batch_size=batch_size), epochs=epochs,
                            steps_per_epoch=len(x_train_arg) // batch_size,
                            validation_data=(x_val_arg, y_val_arg), verbose=2)

In [ ]:
test_loss_affect, test_acc_affect = model2.evaluate(x_test_arg, y_test_arg, batch_size=batch_size)
print(f"Test loss: {test_loss_affect:.4f}")
print(f"Test accuracy: {test_acc_affect:.4f}")

### Results 

In [ ]:
plot_acc_loss(history_fer)

In [ ]:
plot_acc_loss(history_fer)

In [ ]:
# TODO: BESPREKK DE RESULTATEN!!!! MAAR EERST MOET HET MODEL GETRAIEND WORDEN

## Conclusion